In [ ]:
import pandas as pd
import numpy as np
import random

# --- Étape 1 : Création de products_data ---
num_products = 50
products_data = []

departments = ["Department_25", "Department_31", "Department_10"]
classes = {"Department_25": ["Class_2570", "Class_2571"], "Department_31": ["Class_3123", "Class_3124"], "Department_10": ["Class_1050"]}
subclasses = {cls: [f"SubClass_{i}" for i in range(10000, 10005)] for cls in sum(classes.values(), [])}
brands = ["Brand_A", "Brand_B", "Brand_C", "Brand_D"]

for i in range(num_products):
    department = random.choice(departments)
    class_key = random.choice(classes[department])
    subclass_key = random.choice(subclasses[class_key])
    brand = random.choice(brands)
    product_id = f"Product_{10000 + i}"
    description = f"{random.randint(100, 1000)}G {random.choice(['Biscuits', 'Pâtes', 'Savons'])}"
    products_data.append({
        "product_id": product_id,
        "product_description": description,
        "department_key": department,
        "class_key": class_key,
        "subclass_key": subclass_key,
        "sector": "PGC",
        "brand_key": brand,
        "shelf_level1": department,
        "shelf_level2": class_key,
        "shelf_level3": subclass_key,
        "shelf_level4": None,
        "bio": random.choice([0, 1]),
        "sugar_free": random.choice([0, 1]),
        "gluten_free": random.choice([0, 1]),
        "vegan": random.choice([0, 1]),
        "lactose_free": random.choice([0, 1]),
        "carrefour_brand": random.choice([0, 1])
    })

products_data = pd.DataFrame(products_data)

# --- Étape 2 : Création des préférences clients ---
n_customers = 500
customer_ids = [f"Household_{i}" for i in range(1, n_customers + 1)]

# Associer des clients à leurs produits préférés
customer_preferences = {
    customer: random.sample(products_data['product_id'].tolist(), random.randint(5, 10)) for customer in customer_ids
}

# --- Étape 3 : Générer train_data ---
n_train = 10000
start_date = pd.to_datetime("2022-01-01")
end_date = pd.to_datetime("2023-12-31")

train_data = []
for _ in range(n_train):
    customer_id = random.choice(customer_ids)
    product_id = random.choice(customer_preferences[customer_id])
    date = pd.to_datetime(np.random.randint(start_date.value, end_date.value), unit='ns')
    train_data.append({
        "date": date.strftime('%Y-%m-%d'),
        "transaction_id": f"Transaction_{random.randint(1, 1000000)}",
        "customer_id": customer_id,
        "product_id": product_id,
        "has_loyalty_card": random.choice([0, 1]),
        "store_id": f"Store_{random.randint(1, 10)}",
        "is_promo": random.choice([0, 1]),
        "quantity": random.randint(1, 5),
        "format": random.choice(["DRIVE", "DELIVERY", "PICKUP"]),
        "order_channel": random.choice(["MOBILE_APP", "WEBSITE", "IN_STORE"])
    })

train_data = pd.DataFrame(train_data)

# --- Étape 4 : Générer test_data basé sur train_data ---
n_test = 2000
test_data = []

for _ in range(n_test):
    customer_id = random.choice(customer_ids)
    if random.random() < 0.8:
        product_id = random.choice(customer_preferences[customer_id])
    else:
        product_id = random.choice(products_data['product_id'].tolist())
    test_data.append({
        "transaction_id": f"Transaction_{random.randint(1, 1000000)}",
        "customer_id": customer_id,
        "product_id": product_id
    })

test_data = pd.DataFrame(test_data)

# --- Validation ---
print("Products Data Sample:")
print(products_data.head())
print("\nTrain Data Sample:")
print(train_data.head())
print("\nTest Data Sample:")
print(test_data.head())


Products Data Sample:
      product_id product_description department_key   class_key  \
0  Product_10000          975G Pâtes  Department_10  Class_1050   
1  Product_10001          891G Pâtes  Department_25  Class_2570   
2  Product_10002          165G Pâtes  Department_10  Class_1050   
3  Product_10003          861G Pâtes  Department_10  Class_1050   
4  Product_10004         966G Savons  Department_31  Class_3124   

     subclass_key sector brand_key   shelf_level1 shelf_level2  \
0  SubClass_10002    PGC   Brand_B  Department_10   Class_1050   
1  SubClass_10004    PGC   Brand_A  Department_25   Class_2570   
2  SubClass_10004    PGC   Brand_D  Department_10   Class_1050   
3  SubClass_10004    PGC   Brand_D  Department_10   Class_1050   
4  SubClass_10003    PGC   Brand_C  Department_31   Class_3124   

     shelf_level3 shelf_level4  bio  sugar_free  gluten_free  vegan  \
0  SubClass_10002         None    0           1            0      1   
1  SubClass_10004         None    0 

In [ ]:
!pip install scikit-learn --upgrade

In [ ]:
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical


# Séparer X et y
X = train_data.drop(columns=["product_id"])  # Toutes les colonnes sauf product_id
y = train_data["product_id"]  # La cible (product_id)

# Encodage de la variable cible (product_id)
product_encoder = LabelEncoder()
y_encoded = product_encoder.fit_transform(y)

# Encodage des colonnes catégorielles dans X
categorical_columns = X.select_dtypes(include=["object"]).columns

# Appliquer OneHotEncoder sur les colonnes catégorielles
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore") # sparse was deprecated in 1.2 and replaced with sparse_output
X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_columns]))

# Ajouter les colonnes numériques
numerical_columns = X.select_dtypes(exclude=["object"]).columns
X_encoded = X_encoded.astype(np.float32)

# Conversion en matrices numpy
X_train = X_encoded.to_numpy()
y_train = to_categorical(y_encoded, num_classes=len(product_encoder.classes_))

print(f"Dimensions de X_train : {X_train.shape}")
print(f"Dimensions de y_train : {y_train.shape}")


Dimensions de X_train : (10000, 10471)
Dimensions de y_train : (10000, 50)


In [ ]:
# Encodage des données de test
X_test = test_data[['transaction_id', 'customer_id']]  # Select only the columns present in test_data and used for training

# Get categorical columns present in both train and test data
categorical_columns_test = list(set(categorical_columns) & set(X_test.columns))

# Create missing columns in X_test and fill with a placeholder value (e.g., 0)
# We fill with 0 because the numerical columns are related to the training data and do not exist in the test data.
# Instead of 'missing', we use 0 to represent the absence of these features in the test data.
for col in numerical_columns:
    if col not in X_test.columns:
        X_test[col] = 0

# Create missing columns in X_test and fill with a placeholder value (e.g., 'missing')
for col in categorical_columns:
    if col not in X_test.columns:
        X_test[col] = 'missing'  # Or any other suitable placeholder

# Now you can safely apply the transform
X_test_encoded = pd.DataFrame(encoder.transform(X_test[categorical_columns]))

# Continue with the rest of your code...
x_test_final = pd.concat([X_test_encoded, X_test[numerical_columns].reset_index(drop=True)], axis=1)
X_test_final = x_test_final.to_numpy()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

num_features = X_train.shape[1]  # Nombre de colonnes dans X
num_products = y_train.shape[1]  # Nombre de classes (produits)

model = Sequential()
model.add(Dense(128, input_dim=num_features, activation="relu"))
model.add(Dense(64, activation="relu"))
model.add(Dense(num_products, activation="softmax"))  # Prédictions des produits

model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Entraîner le modèle
model.fit(X_train, y_train, epochs=10, batch_size=32)


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 20ms/step - accuracy: 0.0262 - loss: 3.9030
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 24ms/step - accuracy: 0.1852 - loss: 3.5014
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 20ms/step - accuracy: 0.5897 - loss: 1.9796
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 23ms/step - accuracy: 0.9554 - loss: 0.5621
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 23ms/step - accuracy: 0.9995 - loss: 0.1034
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 20ms/step - accuracy: 1.0000 - loss: 0.0366
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 24ms/step - accuracy: 1.0000 - loss: 0.0198
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 10s 21ms/step - accuracy: 1.0000 - loss: 0.0125
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 19ms/step - accuracy: 1.0000 - loss: 0.0085
Epoch 10/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 7s 22ms/step - accuracy: 1.0000 - loss: 0.0061


In [ ]:
def predict_product(model, top_n=10):
    """
    Génère les produits recommandés pour chaque instance dans X_train.

    Args:
        model: Le modèle entraîné.
        top_n: Le nombre de produits recommandés (par défaut : 10).

    Returns:
        Une liste de produits recommandés pour chaque exemple d'entraînement.
    """
    # Prédire les probabilités pour les données d'entraînement
    prediction = model.predict(X_train)

    # Obtenir les indices des produits les plus probables
    top_products = np.argsort(prediction, axis=1)[:, -top_n:][:, ::-1]

    # Décoder les produits pour retourner les identifiants originaux
    return [product_encoder.inverse_transform(top_products[i]) for i in range(len(top_products))]


In [ ]:
# Générer les recommandations
predicted_products = predict_product(model)


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


In [ ]:
def hitrate_at_k(test_data, predicted_products, k=10):
    """
    Calcule le Hitrate@K.

    Args:
        test_data: Le jeu de données de test contenant les produits réellement achetés.
        predicted_products: Liste des produits recommandés par le modèle.
        k: Le nombre de recommandations à considérer (par défaut : 10).

    Returns:
        Le score Hitrate@K.
    """
    hits = 0

    for i, true_product in enumerate(test_data["product_id"]):
        if true_product in predicted_products[i][:k]:  # Vérifier si le vrai produit est dans les top K recommandations
            hits += 1

    return hits / len(test_data)


# Calcul du Hitrate@10
hitrate = hitrate_at_k(test_data, predicted_products, k=10)
print(f"Hitrate@10 : {hitrate:.2f}")


Hitrate@10 : 0.21
